# Import Modules

In [ ]:
# General ------------------------------------------------------------------------------------------------------------------
import os
import math
import random
import itertools
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Image Augmentation -------------------------------------------------------------------------------------------------------
import imgaug as ia
import imgaug.augmenters as iaa

# Deep Learning ------------------------------------------------------------------------------------------------------------
import tensorflow as tf
from tensorflow.keras import optimizers, initializers, layers, activations, metrics

# Bayesian Optimization ----------------------------------------------------------------------------------------------------
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

# Settings

In [ ]:
# Set random seeds ---------------------------------------------------------------------------------------------------------
rSeed = 0
random.seed(rSeed)
ia.seed(rSeed)
np.random.seed(rSeed)

# Set the split fraction between train and test sets -----------------------------------------------------------------------
splitFraction = 0.7

# Bayesian optimization settings -------------------------------------------------------------------------------------------
bayeOptIterationLimit = 25
# bayeOptIterationLimit = 4

# Deep learning model settings ---------------------------------------------------------------------------------------------
regEpochs = 5

# Functions

In [ ]:
### Convolutional Neural Network Models --------------------------------------------------------------------------------------------------------------
def createModel(lr=-1, beta1=0.9, beta2=0.999, epsilon=None, decay=0.0):
    try: del model
    except:pass
    
    # Create a sequential model --------------------------------------------------------------------------------------------
    model = tf.keras.models.Sequential()
    # Add a convolutional layer --------------------------------------------------------------------------------------------
    model.add(layers.Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='valid', input_shape=(41, 41, 1)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='valid'))
    # Add another convolution layer ----------------------------------------------------------------------------------------
    if convLayers >= 2:
        model.add(layers.Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same'))
        model.add(layers.BatchNormalization())
        model.add(layers.ReLU())
        model.add(layers.MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='valid'))
    # Flatten the output ---------------------------------------------------------------------------------------------------
    model.add(layers.Flatten())
    # Add dense layers -----------------------------------------------------------------------------------------------------
    model.add(layers.Dense(2))
    if denseLayers >= 2: 
        model.add(layers.Dense(2))
    if denseLayers >= 3:
        model.add(layers.Dense(2))
    # Add a softmex layer --------------------------------------------------------------------------------------------------
    model.add(layers.Softmax())
    # Set ADAM as the training algorithm -----------------------------------------------------------------------------------
    opt = optimizers.Adam(lr=10**lr, beta_1=beta1, beta_2=beta2, epsilon=epsilon, decay=decay)
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=[metrics.CategoricalAccuracy()])
    
    # Print summary of the model -------------------------------------------------------------------------------------------
#     print(model.inputs, model.outputs, model.summary(), sep="\n\n")
    return model
### --------------------------------------------------------------------------------------------------------------------------------------------------



### Image Augmentation -------------------------------------------------------------------------------------------------------------------------------
def augmentImages(origImages):
    seqAug = iaa.Sequential([iaa.OneOf([iaa.AdditiveGaussianNoise(loc=0, scale=(0, 0.1*255)), iaa.GaussianBlur(sigma=(0, 1))]),
                            iaa.Affine(rotate=(-25, 25))])
    augImages = seqAug.augment_images(origImages)
    return augImages
### --------------------------------------------------------------------------------------------------------------------------------------------------



### Bayesian Optimization ----------------------------------------------------------------------------------------------------------------------------
def objectiveFunction(lr=-4, beta1=0.9, beta2=0.999):
    # Create the model using specific hyperparameters
    model = createModel(lr=lr, beta1=beta1, beta2=beta2)
    # Train the model
    model.fit_generator(generator=trainBatchIterator, steps_per_epoch=len(trainBatchIterator), epochs=regEpochs, verbose=0)
    # Evaluate model
    modelScore = model.evaluate(trainX, categoricalTrainY)
    # Return accuracy on train set
    return modelScore[1]


def bayeOpt():
    # Set region to probe ------------------------------------------------------------------------------------------------------
    pbounds = {"lr": (-6, 0), "beta1": (0.0001, 0.9999), "beta2": (0.0001, 0.9999)}
    # Create the bayesian optimizer --------------------------------------------------------------------------------------------
    optimizer = BayesianOptimization(f=objectiveFunction, pbounds=pbounds, random_state=rSeed,)
    utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.0)
    # Maximize the accuracy ----------------------------------------------------------------------------------------------------
    for _ in range(bayeOptIterationLimit):
        nextPoint = optimizer.suggest(utility)
        target = objectiveFunction(**nextPoint)
        optimizer.register(params=nextPoint, target=target)
#         print(target, nextPoint, end="\n\n")
    return optimizer.max
### --------------------------------------------------------------------------------------------------------------------------------------------------


### Metrics ------------------------------------------------------------------------------------------------------------------------------------------
def evaluateModel(convLayers, denseLayers, setName, trueY, predY):
    tp = tf.keras.metrics.TruePositives()
    tn = tf.keras.metrics.TrueNegatives()
    fp = tf.keras.metrics.FalsePositives()
    fn = tf.keras.metrics.FalseNegatives()
    _ = tp.update_state(trueY, predY)
    _ = tn.update_state(trueY, predY)   
    _ = fp.update_state(trueY, predY)
    _ = fn.update_state(trueY, predY)    
    
    # True Positives -------------------------------------------------------------------------------------------------------
    truePos = tp.result().numpy()
    # True Negatives -------------------------------------------------------------------------------------------------------
    trueNeg = tn.result().numpy()
    # False Positives ------------------------------------------------------------------------------------------------------
    falsePos = fp.result().numpy()
    # False Negatives ------------------------------------------------------------------------------------------------------
    falseNeg = fn.result().numpy()
    
    # Accuracy -------------------------------------------------------------------------------------------------------------
    accuracy = (truePos + trueNeg)/(truePos + trueNeg + falsePos + falseNeg)
    # Sensitivity ----------------------------------------------------------------------------------------------------------
    sensitivity = (truePos)/(truePos + falseNeg)
    # Specificity ----------------------------------------------------------------------------------------------------------
    specificity = (trueNeg)/(trueNeg + falsePos)
    # Precision ------------------------------------------------------------------------------------------------------------
    precision = (truePos)/(truePos + falsePos)
    return convLayers, denseLayers, setName, accuracy, sensitivity, specificity, precision
### --------------------------------------------------------------------------------------------------------------------------------------------------

# Create Train/Test Sets

In [ ]:
# Load dataset -------------------------------------------------------------------------------------------------------------
dataX = scipy.io.loadmat("dataset_41_41_1_13031.mat")["Input"]
dataY = scipy.io.loadmat("dataset_41_41_1_13031.mat")["Target"]

trainX, trainY, testX, testY = [], [], [], []
# Create train and test splits ---------------------------------------------------------------------------------------------
for i in range(dataX.shape[3]):
    if i < math.ceil(dataX.shape[3] * splitFraction):
        trainX.append(dataX[:, :, :, i])
        trainY.append(dataY[:, i])
    else:
        testX.append(dataX[:, :, :, i])
        testY.append(dataY[:, i])
trainX = np.asarray(trainX)
trainY = np.reshape(np.asarray(trainY), (-1,))
testX = np.asarray(testX)
testY = np.reshape(np.asarray(testY), (-1,))

# Augment train set images -------------------------------------------------------------------------------------------------
trainX = np.concatenate((augmentImages(trainX), trainX))
trainY = np.concatenate((trainY, trainY))

# Print shapes of train/test sets ------------------------------------------------------------------------------------------
print("Shape of train data:", trainX.shape)
print("Shape of train labels:", trainY.shape)
print("Shape of test data:", testX.shape)
print("Shape of test label:", testY.shape)


# Convert binary labels into categorical labels ----------------------------------------------------------------------------
categoricalTrainY = np.asarray(tf.keras.utils.to_categorical(trainY, num_classes=2))

# Create image iterator objects --------------------------------------------------------------------------------------------
imgGen = tf.keras.preprocessing.image.ImageDataGenerator()
trainBatchIterator = imgGen.flow(x=trainX, y=categoricalTrainY, batch_size=64, shuffle=True, seed=rSeed)

# Preview Samples

In [ ]:
index = 18
plt.imshow(trainX[index, :, :, 0])
print("This sample belongs to the class:", categoricalTrainY[index])

# Compare Structures

In [ ]:
# CNN Structures to compare: (number of convolution layers, number of dense layers) ----------------------------------------
modelStructures = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3)]

# Evaluate CNN Structures --------------------------------------------------------------------------------------------------
resultsList = []
for i, (convLayers, denseLayers) in enumerate(modelStructures):
    print("\nEvaluating Model: {Convolution layers: ",convLayers, ",", " Dense Layers: ", denseLayers, "} ---------------------------------------------------------------------",sep="")
    # Perform Bayesian optimization ----------------------------------------------------------------------------------------
    bayeOptParameters = bayeOpt()
    # Create the model using the optimum hyper-paramters -------------------------------------------------------------------
    model = createModel(lr=bayeOptParameters["params"]["lr"], beta1=bayeOptParameters["params"]["beta1"], beta2=bayeOptParameters["params"]["beta2"])
    # Train the model ------------------------------------------------------------------------------------------------------
    model.fit_generator(generator=trainBatchIterator, steps_per_epoch=len(trainBatchIterator), epochs=regEpochs, verbose=0)
    # Evaluate the model ---------------------------------------------------------------------------------------------------
    resultsList.append(evaluateModel(convLayers, denseLayers, "Train Set", trainY, np.argmax(model.predict(trainX), axis = 1)))
    resultsList.append(evaluateModel(convLayers, denseLayers, "Test Set", testY, np.argmax(model.predict(testX), axis = 1)))

In [ ]:
# Compile results ----------------------------------------------------------------------------------------------------------
resultsDF = pd.DataFrame(resultsList, columns = ["Number of Convolution Layers", "Number of Dense Layers", "Set", "Accuracy", "Sensitivity", "Specificity", "Precision"])
print(resultsDF)